In [ ]:
!git clone https://github.com/hotoku/python_asr/

Cloning into 'python_asr'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 209 (delta 43), reused 63 (delta 27), pack-reused 128
Receiving objects: 100% (209/209), 279.71 KiB | 12.16 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
!mkdir -p python_asr/data/original
!cp /content/drive/MyDrive/work/20220811-jsut_data/jsut-data.zip python_asr/data/original

In [ ]:
!apt install sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 19 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com/ubun

In [ ]:
!pip install sox numpy matplotlib pyyaml torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

In [ ]:
!ls python_asr

00prepare	    03gmm_hmm  06rnn_attention	data	   README.md
01compute_features  04dnn_hmm  07ctc_att_mtl	LICENSE    requirements.txt
02dp_matching	    05ctc      appendix.pdf	notebooks


In [ ]:
os.chdir("python_asr/00prepare")

In [ ]:
!ls

00download_data.py  01prepare_wav.py  02prepare_label.py  03subset_data.py


In [ ]:
!python 00download_data.py

start: 2022-08-16 02:06:24.639452
jsut-data archive already exists. Skip downloading.
extract jsut-data start
extract jsut-data finished
download jsut-label start
download jsut-label finished
extract jsut-label start
extract jsut-label finished
all processes finished
end: 2022-08-16 02:07:07.566560
elapsed: 0:00:42.927108


In [ ]:
!python 01prepare_wav.py

finished 500/5000
finished 1000/5000
finished 1500/5000
finished 2000/5000
finished 2500/5000
finished 3000/5000
finished 3500/5000
finished 4000/5000
finished 4500/5000
finished 5000/5000


In [ ]:
!python 02prepare_label.py
!python 03subset_data.py

In [ ]:
!ls ..

00prepare	    03gmm_hmm  06rnn_attention	data	   README.md
01compute_features  04dnn_hmm  07ctc_att_mtl	LICENSE    requirements.txt
02dp_matching	    05ctc      appendix.pdf	notebooks


In [ ]:
os.chdir("../01compute_features")

In [ ]:
!python 01_compute_fbank.py
!python 01_compute_mfcc.py
!python 02_compute_mean_std.py

Input wav_scp: ../data/label/train_small/wav.scp
Output directory: ./fbank/train_small
Input wav_scp: ../data/label/train_large/wav.scp
Output directory: ./fbank/train_large
Input wav_scp: ../data/label/dev/wav.scp
Output directory: ./fbank/dev
Input wav_scp: ../data/label/test/wav.scp
Output directory: ./fbank/test
Input wav_scp: ../data/label/train_small/wav.scp
Output directory: ./mfcc/train_small
Input wav_scp: ../data/label/train_large/wav.scp
Output directory: ./mfcc/train_large
Input wav_scp: ../data/label/dev/wav.scp
Output directory: ./mfcc/dev
Input wav_scp: ../data/label/test/wav.scp
Output directory: ./mfcc/test
Input feat_scp: ./fbank/train_small/feats.scp
Output file: ./fbank/train_small/mean_std.txt
Input feat_scp: ./fbank/train_large/feats.scp
Output file: ./fbank/train_large/mean_std.txt
Input feat_scp: ./mfcc/train_small/feats.scp
Output file: ./mfcc/train_small/mean_std.txt
Input feat_scp: ./mfcc/train_large/feats.scp
Output file: ./mfcc/train_large/mean_std.txt


In [ ]:
os.chdir("../05ctc")

In [ ]:
!python 01_get_token.py large

In [ ]:
!python 02_train_ctc.py large phone

MyCTCModel(
  (encoder): Encoder(
    (rnn): ModuleList(
      (0): GRU(40, 320, batch_first=True, bidirectional=True)
      (1): GRU(320, 320, batch_first=True, bidirectional=True)
      (2): GRU(320, 320, batch_first=True, bidirectional=True)
      (3): GRU(320, 320, batch_first=True, bidirectional=True)
      (4): GRU(320, 320, batch_first=True, bidirectional=True)
    )
    (proj): ModuleList(
      (0): Linear(in_features=640, out_features=320, bias=True)
      (1): Linear(in_features=640, out_features=320, bias=True)
      (2): Linear(in_features=640, out_features=320, bias=True)
      (3): Linear(in_features=640, out_features=320, bias=True)
      (4): Linear(in_features=640, out_features=320, bias=True)
    )
  )
  (out): Linear(in_features=320, out_features=38, bias=True)
)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, wh

In [ ]:
!time echo tar czvf /content/drive/MyDrive/work/20220816-python_asr/$(date +%Y%m%d%H%M%S)-exp_train_large_.tar.gz exp_train_large

tar czvf exp_train_large_20220816024545.tar.gz exp_train_large

real	0m0.004s
user	0m0.002s
sys	0m0.002s
